# Figure 1: OAX events in FOSI, Obs and SMYLE

In [ ]:
# general use packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np

# packages for altering time to match up!
import sys
import cftime

# climpred packages
import climpred
from climpred import HindcastEnsemble
from climpred.tutorial import load_dataset
from climpred.stats import rm_poly

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
def detrend_linear(dat, dim, deg):
    """ linear detrend dat along the axis dim """
    params = dat.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(dat[dim], params.polyfit_coefficients)
    dat = dat-fit
    return dat

In [ ]:
ds = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')

ds = ds.groupby('time.month') - ds.groupby('time.month').mean()

ds_omega = detrend_linear(ds.omega_ar,dim='time',deg=1)
ds_temp = detrend_linear(ds.temperature,dim='time',deg=2)

level = 0.1
obs_omega_thold = ds_omega.quantile(level)

level = 0.9
obs_temp_thold = ds_temp.quantile(level)

omega_extreme = ds_omega.where(ds_omega < obs_omega_thold)
temp_extreme = ds_temp.where(ds_temp > obs_temp_thold)

In [ ]:
overlap = (~np.isnan(omega_extreme)).astype('int') + (~np.isnan(temp_extreme)).astype('int')
# overlap = overlap.where(overlap > 1)

In [ ]:
# Blob example
levels = np.arange(-2,2,0.1)
f, ax = plt.subplots(1,1, subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))
im = ds_temp.sel(time='2014-07').squeeze().plot.contourf(ax = ax, levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
temp_extreme.sel(time='2014-07').squeeze().plot.contourf(ax = ax,hatches=["///"],alpha=0,cmap='coolwarm',transform = ccrs.PlateCarree(),add_colorbar=False)
# np.isnan(temp_extreme.sel(time='2014-07').squeeze()).plot.contour(colors='black',alpha=0.7,linewidths=1,transform = ccrs.PlateCarree())

ax.set_extent([-170,-110,20,70], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, color='k', zorder=3)

plt.colorbar(im)
plt.title('')

f.savefig('example_mhw.pdf')

In [ ]:
# La Nina example
# Blob example
levels = np.arange(-0.5,0.5,0.01)

f, ax = plt.subplots(1,1, subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))
im = ds_omega.sel(time='2000-01').squeeze().plot.contourf(ax = ax, levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
# np.isnan(omega_extreme.sel(time='2000-01').squeeze()).plot.contour(colors='black',alpha=0.8,linewidths=1,transform = ccrs.PlateCarree())
omega_extreme.sel(time='2000-01').squeeze().plot.contourf(ax = ax,hatches=["///"],alpha=0,cmap='coolwarm',transform = ccrs.PlateCarree(),add_colorbar=False)
ax.set_extent([-130,-70,-20,30], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, color='k', zorder=3)
plt.title('')
plt.colorbar(im)

f.savefig('example_oax.pdf')

## Look at SMYLE versus obs

In [ ]:
# SMYLE Utility functions
from SMYLEutils import io_utils as io
from SMYLEutils import calendar_utils as cal
from SMYLEutils import stat_utils as stat

In [ ]:
%%time
var = 'omega_arag'; depth = 'surface'
var2 = 'CO3'
smyle08 = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+  var + '.monthly.' + depth + '.08.regrid.nc')[var]
smyle08_time = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.08.time.nc')#.sel(L=slice(1,12))

smyle08_anom,smyle08_clim = stat.remove_drift(smyle08,smyle08_time.time,1982,2019)
smyle08_anom = detrend_linear(smyle08_anom,'Y')

smyle_omega_full = smyle08_anom
smye_omega_time_full = smyle08_time

In [ ]:
smyle_omega_extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.08.binary.Rolling.full.2.nc')['binary']

## Load extremes for hatching

In [ ]:
# rolling 
# var = 'omega_arag'
obs_extremes = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/' + var + '.obs.rolling.thold.Rolling.nc')['threshold']

In [ ]:
obs_extremes = obs_extremes.where(obs_extremes > 0)

In [ ]:
var = 'omega_arag'
var2 = 'CO3'
depth = 'surface'

smyle08_ex = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/thresholds/'+var +  '.monthly.' + depth + '.08.binary.Rolling.full.2.nc')['binary']
smyle08_time_ex = xr.open_dataset('/glade/scratch/smogen/SMYLE-Extreme/'+var2+'.monthly.08.time.nc')#.sel(L=slice(1,12))

In [ ]:
smyle08_ex = smyle08_ex.where(smyle08_ex > 0)

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
## Sept. 1999
smyle_test = smyle_omega_full.isel(L=1).mean('M')#.isel(M=which_M)
smyle_test['Y'] = smyle_test['time']

smyle08_test_ex = smyle08_ex.isel(L=1).mean('M')#.isel(M=which_M)


In [ ]:
f, ax = plt.subplots(3,2,figsize=(7,8),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

which_M = 11
levels = np.arange(-0.3,0.35,0.05)

## Sept. 1999
smyle_test = smyle_omega_full.isel(L=1).mean('M')#.isel(M=which_M)
smyle_test['Y'] = smyle_test['time']

smyle08_test_ex = smyle08_ex.isel(L=1).fillna(0).mean('M')#.isel(M=which_M)
smyle08_test_ex = smyle08_test_ex.where(smyle08_test_ex > 0.5)
# smyle08_test_ex['Y'] = smyle08_test_ex['time']

ds_omega.sel(time='1999-09').squeeze().plot.contourf(ax = ax[0,0], levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
obs_extremes.sel(time='1999-09').squeeze().plot.contourf(ax = ax[0,0],hatches=["///"],alpha=0,cmap='coolwarm',transform = ccrs.PlateCarree(),add_colorbar=False)
ds_sept_corr = xr.corr(ds_omega.sel(time='1999-09').squeeze(),obs_extremes.sel(time='1999-09').squeeze())
ax[0,0].set_title('1999-09 Obs')
ax[0,0].set_extent([-180,-70,-20,70], crs=ccrs.PlateCarree())
ax[0,0].add_feature(cfeature.LAND, color='k', zorder=3)

smyle_test.sel(Y='1999').squeeze().plot.contourf(ax = ax[0,1], levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
smyle08_test_ex.sel(Y=1999).plot.contourf(ax = ax[0,1],alpha=0,hatches=["///"],cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
ex_sept_corr = xr.corr(smyle_test.sel(Y='1999').squeeze(),smyle08_test_ex.sel(Y=1999))
ax[0,1].set_title('1.5 Lead 1999-09 SMYLE')
ax[0,1].set_extent([-180,-70,-20,70], crs=ccrs.PlateCarree())
ax[0,1].add_feature(cfeature.LAND, color='k', zorder=3)

## Nov. 1999
smyle_test = smyle_omega_full.isel(L=3).mean('M')#.isel(M=which_M)
smyle_test['Y'] = smyle_test['time']

smyle08_test_ex = smyle08_ex.isel(L=3).fillna(0).mean('M')#.isel(M=which_M)
smyle08_test_ex = smyle08_test_ex.where(smyle08_test_ex > 0.5)
# smyle08_test_ex['Y'] = smyle08_test_ex['time']

ds_omega.sel(time='1999-11').squeeze().plot.contourf(ax = ax[1,0], levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
obs_extremes.sel(time='1999-11').squeeze().plot.contourf(ax = ax[1,0],hatches=["///"],alpha=0,cmap='coolwarm',transform = ccrs.PlateCarree(),add_colorbar=False)
ds_nove_corr = xr.corr(ds_omega.sel(time='1999-11').squeeze(),obs_extremes.sel(time='1999-11').squeeze())
ax[1,0].set_title('1999-09 Obs')
ax[1,0].set_extent([-180,-70,-20,70], crs=ccrs.PlateCarree())
ax[1,0].add_feature(cfeature.LAND, color='k', zorder=3)

smyle_test.sel(Y='1999').squeeze().plot.contourf(ax = ax[1,1], levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
smyle08_test_ex.sel(Y=1999).plot.contourf(ax = ax[1,1],alpha=0,hatches=["///"],cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
ex_nove_corr = xr.corr(smyle_test.sel(Y='1999').squeeze(),smyle08_test_ex.sel(Y=1999))
ax[1,1].set_title('3.5 Lead 1999-11 SMYLE')
ax[1,1].set_extent([-180,-70,-20,70], crs=ccrs.PlateCarree())
ax[1,1].add_feature(cfeature.LAND, color='k', zorder=3)

## Jan. 2000
smyle_test = smyle_omega_full.isel(L=5).mean('M')#.isel(M=which_M)
smyle_test['Y'] = smyle_test['time']

smyle08_test_ex = smyle08_ex.isel(L=5).fillna(0).mean('M')#.sel(M=which_M)
smyle08_test_ex = smyle08_test_ex.where(smyle08_test_ex > 0.5)
# smyle08_test_ex['Y'] = smyle08_test_ex['time']

ds_omega.sel(time='2000-01').squeeze().plot.contourf(ax = ax[2,0], levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
obs_extremes.sel(time='2000-01').squeeze().plot.contourf(ax = ax[2,0],hatches=["///"],alpha=0,cmap='coolwarm',transform = ccrs.PlateCarree(),add_colorbar=False)
ds_janu_corr = xr.corr(ds_omega.sel(time='2000-01').squeeze(),obs_extremes.sel(time='2000-01').squeeze())
ax[2,0].set_title('2000-01 Obs')
ax[2,0].set_extent([-180,-70,-20,70], crs=ccrs.PlateCarree())
ax[2,0].add_feature(cfeature.LAND, color='k', zorder=3)

im = smyle_test.sel(Y='2000').squeeze().plot.contourf(ax = ax[2,1], levels=levels,cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
smyle08_test_ex.sel(Y=2000).plot.contourf(ax = ax[2,1],alpha=0,hatches=["///"],cmap='coolwarm',transform = ccrs.PlateCarree(), extend='both',add_colorbar=False)
ex_janu_corr = xr.corr(smyle_test.sel(Y='2000').squeeze(),smyle08_test_ex.sel(Y=2000))
ax[2,1].set_title('5.5 Lead 2000-01 SMYLE')
ax[2,1].set_extent([-180,-70,-20,70], crs=ccrs.PlateCarree())
ax[2,1].add_feature(cfeature.LAND, color='k', zorder=3)

# f.suptitle('Sample $\Omega_{arag}$ Forecast')

f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.83, 0.25, 0.025, 0.50])
cbar = f.colorbar(im, cax=cbar_ax, ticks=[-0.3,-0.2,-0.1,0,0.1,0.2,0.3])
cbar.ax.tick_params(labelsize=8)
cbar.set_label('$\Omega_{arag} anomaly$', rotation=270,fontsize=10)

f.savefig('./figures/Figure1.example.pdf')
# f.savefig('./figures/Figure1.example.eps')